<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/new_training_CNN_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traininig CNN Spectrum

# Loading Necesary Libraries

In [1]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Processing

In [4]:
input_path = path + "/Spectrum/CNN/working_df.csv"

In [5]:
working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

In [6]:
working_df

,row,PBRS_id,Channels,Distance_km,power_dBm,#span,-32.15625,-32.125,-32.09375,-32.0625,...,32.125,32.15625,32.1875,32.21875,32.25,32.28125,32.3125,32.34375,32.375,32.40625
0,1,1,1,80,0,1,-14.851402,-21.638525,-23.529105,-14.969324,...,-19.957216,-19.447732,-14.576655,-15.491358,-14.654316,-17.801658,-16.387551,-13.980891,-15.122689,-17.753853
1,2,1,1,160,0,2,-32.415520,-23.442737,-17.642596,-16.382163,...,-15.348137,-13.125596,-15.424900,-18.560355,-17.197693,-12.466164,-12.735882,-10.007690,-13.820587,-21.908636
2,3,1,1,240,0,3,-23.428437,-21.738735,-19.956032,-18.320372,...,-21.210045,-20.014100,-13.636612,-9.728106,-7.977970,-19.904029,-22.658427,-19.765777,-24.603827,-31.921519
3,4,1,1,320,0,4,-14.298403,-15.210905,-15.951684,-16.209202,...,-18.673789,-12.470911,-12.638057,-9.282286,-12.488977,-10.933173,-15.749634,-15.676254,-15.412170,-14.347517
4,5,1,1,400,0,5,-19.455200,-19.755724,-18.313774,-13.272877,...,-17.697814,-16.074418,-16.232349,-13.788018,-22.471867,-23.016590,-22.392253,-18.544159,-18.765642,-23.196090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,321,13,1,1680,0,21,-31.466739,-32.655263,-31.882492,-27.589879,...,-34.359245,-32.466935,-29.543434,-31.321699,-33.322457,-34.343455,-30.096108,-33.185712,-33.367392,-33.876192
321,322,13,1,1760,0,22,-21.303913,-33.080588,-37.034437,-31.766035,...,-32.977742,-28.624796,-27.495186,-23.847968,-25.561252,-34.242901,-40.735728,-34.014182,-33.957388,-35.627786
322,323,13,1,1840,0,23,-27.601906,-27.758137,-27.102083,-27.319948,...,-33.646975,-25.620092,-34.883694,-42.119980,-35.870338,-28.109018,-32.237494,-32.837088,-31.096978,-25.976283
323,324,13,1,1920,0,24,-31.725625,-38.937830,-39.113483,-26.640446,...,-21.790284,-24.651905,-26.129703,-26.548903,-25.567147,-30.324506,-30.829273,-33.027456,-34.010274,-34.402337


In [7]:
def rand(start, end, num): # get random value function
    res = []
    for j in range(num):
        res.append(np.random.randint(start, end))
    return res

In [8]:
def calc_freq_behavior(working_df, selected_point, multi=False): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results

  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [9]:
def select_eval_cols(working_df, eval_cols_param): # retturn index of column to be evaluated
  if multi_params[0] == 'even': 
    break_point = int((working_df.shape[1]-6)/(eval_cols_param[1]+1))
    selected_cols = []
    for col_idx in range(1, eval_cols_param[1]+1):
      selected_cols.append((col_idx*break_point)-1)
  else:
    selected_cols = rand(1, working_df.shape[1]-6, eval_cols_param[1])
    selected_cols = sorted(selected_cols)
  return selected_cols

In [10]:
# multi_params = (method, number_freq_to_analyze)
def multiple_freq_analyzer(working_df, multi_params): #method = 'rand' or 'even' (multiple freq behavior calculator)
  selected_cols = select_eval_cols(working_df, multi_params)

  multi_freq_behavior = {}
  for selected_point in tqdm(selected_cols):
    freq_behavior = calc_freq_behavior(working_df, selected_point, multi=True)
    selected_v = [v for v in freq_behavior.keys()]

    selected_d = {}
    for data in freq_behavior.values():
      for k, v in data.items():
        selected_d[k] = v

    multi_freq_behavior[selected_v[0]] = selected_d
  return multi_freq_behavior

In [11]:
multi_params = ('even', 3)
multi_freq_behavior = multiple_freq_analyzer(working_df, multi_params)

100%|██████████| 3/3 [00:00<00:00, 44.75it/s]


In [12]:
multi_freq_behavior.keys()

dict_keys([-16.0625, 0.0625, 16.1875])

In [13]:
def freq_dict_to_df(multi_freq_behavior):
  distances = [x*80 for x in range(1, 26)]
  freq_mean_data = {'distances':distances}
  freq_std_data = {'distances':distances}

  for header, distances_dict in multi_freq_behavior.items():
    col_data_mean = []
    col_data_std = []
    for distance, results in distances_dict.items():
      col_data_mean.append(results[0])
      col_data_std.append(results[1])
    freq_mean_data[header] = col_data_mean
    freq_std_data[header] = col_data_std

  freq_mean_df = pd.DataFrame(freq_mean_data)
  freq_std_df = pd.DataFrame(freq_std_data)
  
  return freq_mean_df, freq_std_df

# Create dfs with freqs selected and resulting freqs

In [14]:
eval_cols_param = ('even', 3) # (evenly separate points(other option 'rand'for randomly selected points), mount of points to be selected)
eval_points = select_eval_cols(working_df, eval_cols_param) # get indexes of of points to be evaluated

real_eval_idx = [eval_point+6 for eval_point in eval_points]

extra_data = working_df.iloc[:, 0:6]
eval_freq_df = working_df.iloc[:, real_eval_idx]
eval_freq_data_df = pd.merge(extra_data, eval_freq_df, left_index=True, right_index=True) #df we only eval freqs

remaining_working_df= working_df.copy()
remaining_working_df.drop(remaining_working_df.columns[real_eval_idx], axis=1, inplace=True) # df without eval freqs

In [15]:
eval_freq_data_df

,row,PBRS_id,Channels,Distance_km,power_dBm,#span,-16.0625,0.0625,16.1875
0,1,1,1,80,0,1,-15.358299,-8.588682,-12.413894
1,2,1,1,160,0,2,-6.339264,-17.555061,-9.442121
2,3,1,1,240,0,3,-11.278372,-16.228628,-11.012252
3,4,1,1,320,0,4,-8.849275,-10.287671,-7.145525
4,5,1,1,400,0,5,-14.922281,-14.450624,-7.644564
...,...,...,...,...,...,...,...,...,...
320,321,13,1,1680,0,21,-13.090786,-8.934059,-8.024277
321,322,13,1,1760,0,22,-17.799330,-7.998582,-8.975237
322,323,13,1,1840,0,23,-9.081565,-12.306357,-8.698755
323,324,13,1,1920,0,24,-8.340772,-12.060949,-15.145052


# Pearson Correletion

In [16]:
#eval_method = 'avg' or 'min' or 'max'

def pearson_matrix(working_df , source_sample_id, eval_method = 'avg'):
  distances = [x*80 for x in range(1, 26)]
  result_data = {'source/target': distances}
  
  print('Generating Pearson Confusion Matrix using ({}) method:'.format(eval_method))
  for target_dist in tqdm(distances): # loop all distances available
    result_target = []
    for source_dist in distances: # loop all distances available
      source_df = working_df[(working_df['Distance_km'] == source_dist) & (working_df['PBRS_id'] == source_sample_id)]# select data of source (distance and sample_id) 
      source_data = source_df.iloc[0, 6:source_df.shape[1]].tolist()
      pearson_values = []
      for target_sample_id in working_df['PBRS_id'].unique(): # loop all samples id to then exclude the source sample id
        if target_sample_id != source_sample_id:
          target_df = working_df[(working_df['Distance_km'] == target_dist) & (working_df['PBRS_id'] == target_sample_id)] # select data of target
          target_data = target_df.iloc[0, 6:target_df.shape[1]].tolist()
          pearson_values.append(round(pearsonr(source_data, target_data)[0], 5)) # pearson correlation
      if eval_method == 'avg':
        p_result = np.mean(pearson_values)
      elif eval_method == 'max':
        p_result = np.max(pearson_values)
      elif eval_method == 'min':
        p_result = np.min(pearson_values)
      result_target.append(p_result)
    result_data[str(eval_method)+'_'+str(target_dist)] = result_target

  pearson_matrix_df = pd.DataFrame.from_dict(result_data)
  #pearson_matrix_df = pearson_matrix_df.set_index('source/target')

  return pearson_matrix_df, result_data

In [17]:
methods = ['min', 'avg', 'max']

dfs = []
for m in methods:
  pearson_matrix_df, result_data = pearson_matrix(eval_freq_data_df , source_sample_id=1, eval_method = m)
  dfs.append(pearson_matrix_df)

final_df = reduce(lambda  left,right: pd.merge(left,right,on=['source/target'],how='outer'), dfs)

Generating Pearson Confusion Matrix using (min) method:


100%|██████████| 25/25 [00:08<00:00,  2.79it/s]


Generating Pearson Confusion Matrix using (avg) method:


100%|██████████| 25/25 [00:08<00:00,  2.83it/s]


Generating Pearson Confusion Matrix using (max) method:


100%|██████████| 25/25 [00:08<00:00,  2.78it/s]


In [18]:
final_df

,source/target,min_80,min_160,min_240,min_320,min_400,min_480,min_560,min_640,min_720,...,max_1280,max_1360,max_1440,max_1520,max_1600,max_1680,max_1760,max_1840,max_1920,max_2000
0,80,-0.56521,-0.99849,-0.88752,-0.99965,-0.94749,-0.51585,-0.99848,-0.92081,-0.97367,...,0.99993,0.85390,0.05764,0.76896,0.21753,0.99964,0.87372,0.05100,0.78044,0.35392
1,160,-0.99787,-0.91200,-0.18078,-0.90970,-0.44798,-0.99826,-0.92697,-0.24976,-0.88019,...,0.41714,0.99991,0.95180,0.99999,1.00000,0.45863,0.99403,0.97378,0.99125,0.99786
2,240,-0.98288,-0.99437,-0.47792,-0.99376,-0.70400,-0.96879,-0.99769,-0.53874,-0.98419,...,0.11355,0.96400,0.99999,0.95140,0.99970,0.15927,0.96152,0.99939,0.90094,0.99961
3,320,-0.75685,-0.91770,-0.85487,-0.94526,-0.96567,-0.83731,-0.90019,-0.88936,-0.94541,...,0.09398,0.99998,0.95683,0.99994,0.89407,0.21621,0.99557,0.93762,0.99302,0.89212
4,400,-0.90613,-0.58869,-0.99992,-0.99786,-0.99704,-0.99896,-0.55419,-0.99983,-0.99987,...,0.58667,0.86664,0.99997,0.94815,0.99965,0.68219,0.92525,0.99916,0.91529,0.99795
5,480,-0.94793,-0.98438,-0.78323,-0.60840,-0.81464,-0.94572,-0.94781,-0.82222,-0.50247,...,0.99991,0.51797,0.99805,0.99480,0.96569,0.99824,0.50250,0.99992,0.99577,0.93770
6,560,0.04254,-0.99739,-0.99975,-0.89969,-0.98435,0.10107,-0.99146,-0.99966,-0.93974,...,0.95236,0.99541,0.64492,0.99873,0.75949,0.94203,0.99988,0.63982,0.99948,0.84416
7,640,-0.57177,-0.89560,-0.99946,-0.96140,-0.99981,-0.56612,-0.99985,-0.99990,-0.91858,...,0.96322,0.55703,0.99990,0.99681,1.00000,0.97098,0.62197,1.00000,0.99987,0.99997
8,720,0.13611,-0.99977,-0.99889,-0.93669,-0.96346,0.18723,-0.99932,-0.99970,-0.96768,...,0.91954,0.98203,0.71379,0.99905,0.81719,0.90638,0.99402,0.70912,0.99864,0.89074
9,800,-0.38306,-0.94696,-0.99835,-0.99801,-0.98707,-0.37670,-0.97295,-0.97981,-0.98186,...,0.88344,0.72178,0.97980,0.99080,0.99921,0.89736,0.77506,0.97845,0.98893,0.99997


# Mean/std analysis

In [19]:
def calc_freq_behavior(working_df, selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results
  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [20]:
test_freq_data_df = calc_freq_behavior(eval_freq_data_df, selected_point=1)

In [21]:
test_freq_data_df

{0.0625: {80: (-8.20918530381542, 2.455948134698787),
  160: (-12.691059594166113, 4.371002582774201),
  240: (-11.610602740090254, 2.518044703307624),
  320: (-10.80222468462674, 3.3434231245597825),
  400: (-11.745395799275535, 2.393004806104668),
  480: (-8.157465803582074, 2.5358125121661907),
  560: (-12.125693854825188, 5.1409637674046875),
  640: (-10.955041500205292, 2.33398183712328),
  720: (-10.366524177856146, 3.6730562644863154),
  800: (-11.409922159977684, 2.4806580142327457),
  880: (-7.822716008373764, 2.6189978546514343),
  960: (-12.603914874827087, 4.6671158597756355),
  1040: (-11.060789724961309, 2.3264185562683113),
  1120: (-11.404392581631903, 3.7267434924718663),
  1200: (-11.484194691633393, 2.3245574027804894),
  1280: (-8.233328288252464, 2.554638442845694),
  1360: (-12.624818555240848, 4.37564766389701),
  1440: (-11.102051215666885, 2.1503681596918893),
  1520: (-10.839646534491177, 3.4003093628609067),
  1600: (-11.632627211672864, 2.303310549275044),
 

In [22]:
def calc_freq_behavior_v2(working_df, amt_of_selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]

  distances = [x*80 for x in range(1, 26)]

  std_combined = []
  for dist in distances: # loop all distances available
    std_all = []
    for val_idx in range(0, amt_of_selected_point):

      freq_selected = working_df.iloc[:, val_idx+6]
      freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
      freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

      headers = list(freq_data_df.columns.values)
      selected_val = round(float(headers[-1]), 5)
      
      distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
      freq_distance_vals = distance_df[str(selected_val)].tolist()

      freq_std = np.std(freq_distance_vals)
      std_all.append(freq_std)
      std_avg = round(np.average(std_all), 5)
    std_combined.append(std_avg)


  data = {'source/target':distances, 'std':std_combined}
 
  # Create DataFrame
  std_df = pd.DataFrame(data)
  return std_df

In [23]:
std_df = calc_freq_behavior_v2(eval_freq_data_df, amt_of_selected_point=multi_params[1])

In [24]:
std_df

,source/target,std
0,80,2.53796
1,160,4.39126
2,240,2.20327
3,320,3.34719
4,400,3.12300
5,480,2.54655
6,560,4.59612
7,640,2.16693
8,720,3.30457
9,800,3.05516


In [25]:
final_final_df = pd.merge(final_df, std_df, on = "source/target", how = "inner")

In [26]:
final_final_df

,source/target,min_80,min_160,min_240,min_320,min_400,min_480,min_560,min_640,min_720,...,max_1360,max_1440,max_1520,max_1600,max_1680,max_1760,max_1840,max_1920,max_2000,std
0,80,-0.56521,-0.99849,-0.88752,-0.99965,-0.94749,-0.51585,-0.99848,-0.92081,-0.97367,...,0.85390,0.05764,0.76896,0.21753,0.99964,0.87372,0.05100,0.78044,0.35392,2.53796
1,160,-0.99787,-0.91200,-0.18078,-0.90970,-0.44798,-0.99826,-0.92697,-0.24976,-0.88019,...,0.99991,0.95180,0.99999,1.00000,0.45863,0.99403,0.97378,0.99125,0.99786,4.39126
2,240,-0.98288,-0.99437,-0.47792,-0.99376,-0.70400,-0.96879,-0.99769,-0.53874,-0.98419,...,0.96400,0.99999,0.95140,0.99970,0.15927,0.96152,0.99939,0.90094,0.99961,2.20327
3,320,-0.75685,-0.91770,-0.85487,-0.94526,-0.96567,-0.83731,-0.90019,-0.88936,-0.94541,...,0.99998,0.95683,0.99994,0.89407,0.21621,0.99557,0.93762,0.99302,0.89212,3.34719
4,400,-0.90613,-0.58869,-0.99992,-0.99786,-0.99704,-0.99896,-0.55419,-0.99983,-0.99987,...,0.86664,0.99997,0.94815,0.99965,0.68219,0.92525,0.99916,0.91529,0.99795,3.12300
5,480,-0.94793,-0.98438,-0.78323,-0.60840,-0.81464,-0.94572,-0.94781,-0.82222,-0.50247,...,0.51797,0.99805,0.99480,0.96569,0.99824,0.50250,0.99992,0.99577,0.93770,2.54655
6,560,0.04254,-0.99739,-0.99975,-0.89969,-0.98435,0.10107,-0.99146,-0.99966,-0.93974,...,0.99541,0.64492,0.99873,0.75949,0.94203,0.99988,0.63982,0.99948,0.84416,4.59612
7,640,-0.57177,-0.89560,-0.99946,-0.96140,-0.99981,-0.56612,-0.99985,-0.99990,-0.91858,...,0.55703,0.99990,0.99681,1.00000,0.97098,0.62197,1.00000,0.99987,0.99997,2.16693
8,720,0.13611,-0.99977,-0.99889,-0.93669,-0.96346,0.18723,-0.99932,-0.99970,-0.96768,...,0.98203,0.71379,0.99905,0.81719,0.90638,0.99402,0.70912,0.99864,0.89074,3.30457
9,800,-0.38306,-0.94696,-0.99835,-0.99801,-0.98707,-0.37670,-0.97295,-0.97981,-0.98186,...,0.72178,0.97980,0.99080,0.99921,0.89736,0.77506,0.97845,0.98893,0.99997,3.05516


In [27]:
final_final_df.to_csv(path+'/Spectrum/CNN/training_data_DNN.csv')